<a href="https://colab.research.google.com/github/slxslxslx/BiShe/blob/main/DDP_BERT_sentiment_pt_2_tqdm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets transformers tqdm scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# 尝试-这个单GPU可以顺利运行吗??

# ds = load_dataset("contemmcm/sentiment140")
# 百万级别1600000，二分类
# 训练集数据量: DatasetDict({
#     complete: Dataset({
#         features: ['text', 'label'],
#         num_rows: 1600000
#     })
# })
# RTX 2080 Ti * 2

import os

# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# import os
#
# os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

import time
from datetime import datetime, timedelta, timezone
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split, DistributedSampler
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from tqdm import tqdm  # 引入 tqdm 用于显示进度条
from sklearn.metrics import classification_report
import numpy as np
import pickle

# 获取北京时区对象
beijing_tz = timezone(timedelta(hours=8))


# 获取当前的UTC时间并转换为北京时间
def get_beijing_time_now():
    return datetime.now(beijing_tz).strftime("%Y-%m-%d %H:%M:%S")


# 加载IMDB数据集和BERT分词器，并进行一些初始化操作
def load_and_preprocess_data():
    # if os.path.exists('encoded_dataset_sentiment140.pt'):
    if True:
        print('load_and_preprocess_data开始执行  if')
        dataset = load_dataset("contemmcm/sentiment140", cache_dir="./dataset/sentiment140")
        print("dataset['complete'][:5]:", dataset['complete'][:5])
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

        # 文本编码函数
        def encode_batch(batch):
            # encoded = tokenizer(batch['text'], padding=True, truncation=True, max_length=128)
            encoded = tokenizer(
                batch['text'],
                padding='max_length',  # 使用 max_length 填充
                truncation=True,
                max_length=128,
                return_tensors='pt'  # 返回 PyTorch 张量
            )
            # print(f"Encoded input_ids lengths: {[len(ids) for ids in encoded['input_ids']]}")
            return encoded

        # 编码数据集
        print('开始编码数据集', get_beijing_time_now())
        encoded_dataset = dataset.map(encode_batch, batched=True)
        encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
        print("encoded_dataset['complete'][:5]:", encoded_dataset['complete'][:5])
        # 12min

        # 保存为通用格式
        torch.save(encoded_dataset["complete"], "encoded_dataset_sentiment140.pt")
        print('保存为 encoded_dataset_sentiment140.pt')
        print('结束编码数据集', get_beijing_time_now())

        # 保存encoded_dataset，这个不好，有问题
        with open('encoded_dataset-sentiment140-original.pkl', 'wb') as f:
            pickle.dump(encoded_dataset, f)
            print('保存encoded_dataset-sentiment140-original.pkl')

            # 划分训练集和测试集
            # 假设encoded_dataset['complete']是你的数据集，这里获取其长度
            total_length = len(encoded_dataset['complete'])
            # 计算训练集的大小（这里按80%比例计算，可根据实际需求修改比例）
            train_size = int(total_length * 0.8)
            # 计算测试集的大小
            test_size = total_length - train_size
            print('total_length', total_length)
            print('train_size', train_size)
            print('test_size', test_size)
            # train_dataset, test_dataset = random_split(encoded_dataset['complete'], [20000, 5000])
            train_dataset, test_dataset = random_split(encoded_dataset['complete'], [train_size, test_size])

            # 使用数据集中自带的train部分作为训练集
            # train_dataset = encoded_dataset['train']
            # test_dataset = encoded_dataset['test']

            test_loader = DataLoader(test_dataset, batch_size=64)
            print('load_and_preprocess_data执行完毕')
            return train_dataset, test_dataset, test_loader

    else:
        print('load_and_preprocess_data开始执行 else')
        # with open('encoded_dataset-sentiment140.pkl', 'rb') as f:
        #     encoded_dataset = pickle.load(f)

        # 使用数据
        encoded_dataset = torch.load("encoded_dataset_sentiment140.pt")
        print('encoded_dataset', encoded_dataset)

        # print(encoded_dataset)

        # 划分训练集和测试集
        # 假设encoded_dataset['complete']是你的数据集，这里获取其长度
        total_length = len(encoded_dataset)
        # 计算训练集的大小（这里按80%比例计算，可根据实际需求修改比例）
        train_size = int(total_length * 0.8)
        # 计算测试集的大小
        test_size = total_length - train_size
        print('total_length', total_length)
        print('train_size', train_size)
        print('test_size', test_size)
        # train_dataset, test_dataset = random_split(encoded_dataset['complete'], [20000, 5000])
        # train_dataset, test_dataset = random_split(encoded_dataset['complete'], [train_size, test_size])
        train_dataset, test_dataset = random_split(encoded_dataset, [train_size, test_size])

        # 使用数据集中自带的train部分作为训练集
        # train_dataset = encoded_dataset['train']
        # test_dataset = encoded_dataset['test']

        test_loader = DataLoader(test_dataset, batch_size=64)
        print('load_and_preprocess_data执行完毕')
        return train_dataset, test_dataset, test_loader


# 单GPU训练
def train_single_gpu(model, train_dataset, epochs=1):
    print('单GPU训练开始,epochs=', epochs)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # 优化器用于根据计算得到的损失值来更新模型的参数
    # 以使得模型能够朝着损失值不断减小的方向进行优化，进而提高模型对数据的拟合能力和预测准确性。
    # 自适应矩估计（Adaptive Moment Estimation）优化算法
    # model.parameters() 获取模型中所有需要被优化的参数

    start_time = time.time()
    print('单GPU start_time', get_beijing_time_now())
    model.train()
    # for epoch in range(epochs):
    #     print('epoch', epoch)
    #     for batch in train_loader:
    #         input_ids = batch['input_ids'].to(device)
    #         attention_mask = batch['attention_mask'].to(device)
    #         labels = batch['label'].to(device)

    #         optimizer.zero_grad()
    #         outputs = model(input_ids, attention_mask=attention_mask)
    #         loss = criterion(outputs.logits, labels)
    #         loss.backward()
    #         optimizer.step()

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs} 开始...")
        # 使用 tqdm 显示进度条
        with tqdm(total=len(train_loader), desc=f"训练进度 (Epoch {epoch + 1}/{epochs})") as pbar:
            for batch_idx, batch in enumerate(train_loader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)
                loss.backward()
                optimizer.step()

                # 更新 tqdm 进度条
                pbar.update(1)
                pbar.set_postfix({"Loss": f"{loss.item():.4f}"})  # 动态显示当前损失值

    end_time = time.time()
    print('单GPU end_time', get_beijing_time_now())
    print(f"Single GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():  # 不进行梯度计算
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print('单GPU评估:\n', classification_report(all_labels, all_preds))


# 2GPU训练
def train_multi_gpu(rank, world_size, model, train_dataset, test_loader, epochs=1):
    # print('test_loader', test_loader)
    print(f"多GPU分布式训练进程 {rank} 初始化")

    # 设置分布式环境变量
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size, timeout=timedelta(seconds=180))

    # 设定设备
    device = torch.device(f"cuda:{rank}")
    model = model.to(device)
    model = DDP(model, device_ids=[rank])
    # device_ids=[rank] 这个参数就是明确告诉 DDP 模块，当前这个模型副本应该放置在哪个具体的 GPU 设备上进行后续的训练操作。保证每个进程所负责的模型副本和相应的 GPU 紧密关联起来

    # 使用 DistributedSampler 确保每个进程处理不同的数据
    train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False, sampler=train_sampler)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    start_time = time.time()
    print(f'多GPU,rank={rank} start_time', get_beijing_time_now())
    if rank == 0:
        print(f"多GPU训练开始时间: {get_beijing_time_now()}")
    model.train()

    for epoch in range(epochs):
        if rank == 0:
            print(f"Epoch {epoch} 开始")
        train_sampler.set_epoch(epoch)
        for batch in train_loader:
            # print(rank, 'batch')
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

    end_time = time.time()
    print(f'多GPU,rank={rank} end_time', get_beijing_time_now())
    if rank == 0:
        print(f"Multi GPU training time: {end_time - start_time:.2f} seconds")

    # 评估模型
    if rank == 0:
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print('多GPU评估:\n', classification_report(all_labels, all_preds))

    dist.destroy_process_group()


# 设置进程启动，通过mp.spawn启动多进程分布式训练
def spawn_train(world_size, model, train_dataset, test_loader):
    mp.spawn(train_multi_gpu, args=(world_size, model, train_dataset, test_loader), nprocs=world_size,
             join=True)  # join=True时mp.spawn 函数会阻塞主进程，直到所有启动的子进程都执行完毕。


# 主函数入口
if __name__ == '__main__':
    train_dataset, test_dataset, test_loader = load_and_preprocess_data()
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数

    # 单GPU训练
    train_single_gpu(model, train_dataset)

    # 多GPU分布式训练!
    # spawn_train(world_size=2, model=model, train_dataset=train_dataset, test_loader=test_loader)

    print('main')

# # 单GPU训练
# def train_single_gpu(model, train_dataset, epochs=1):
#     print('单GPU训练开始,epochs=', epochs)
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
#
#     train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#     optimizer = optim.Adam(model.parameters(), lr=1e-4)  # 优化器用于根据计算得到的损失值来更新模型的参数
#     # 以使得模型能够朝着损失值不断减小的方向进行优化，进而提高模型对数据的拟合能力和预测准确性。
#     # 自适应矩估计（Adaptive Moment Estimation）优化算法
#     # model.parameters() 获取模型中所有需要被优化的参数
#
#     start_time = time.time()
#     print('单GPU start_time', get_beijing_time_now())
#     model.train()
#     for epoch in range(epochs):
#         print('epoch', epoch)
#         for batch in train_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['label'].to(device)
#
#             optimizer.zero_grad()
#             outputs = model(input_ids, attention_mask=attention_mask)
#             loss = criterion(outputs.logits, labels)
#             loss.backward()
#             optimizer.step()
#
#     end_time = time.time()
#     print('单GPU end_time', get_beijing_time_now())
#     print(f"Single GPU training time: {end_time - start_time:.2f} seconds")
#
#     # 评估模型
#     model.eval()
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():  # 不进行梯度计算
#         for batch in test_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['label'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             preds = torch.argmax(outputs.logits, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#
#     print('单GPU评估:\n', classification_report(all_labels, all_preds))


load_and_preprocess_data开始执行  if


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/361 [00:00<?, ?B/s]

polarity.csv:   0%|          | 0.00/124M [00:00<?, ?B/s]

Generating complete split: 0 examples [00:00, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

开始编码数据集 2024-11-21 19:30:50


Map:   0%|          | 0/1600000 [00:00<?, ? examples/s]

KeyboardInterrupt: 